# Instalations

In [ ]:
!pip install --upgrade typing-extensions
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


# GPT APIs

In [ ]:
import openai
import time
import os

def _ms_since_epoch():
    return time.perf_counter_ns() // 1000000


def set_openai_parameters(engine, max_tokens):
    # openai API setup and parameters
    openai.api_key = "sk-proj-LEu3lWTFbZ1inVRqd5E9T3BlbkFJqATqVJMZbDaGnm7nznzH"
    parameters = {
        "max_tokens": max_tokens,
        "top_p": 0,  # greedy
        "temperature": 0.5,
        "logprobs": 5,  # maximal value accorrding to https://beta.openai.com/docs/api-reference/completions/create#completions/create-logprobs, used to be 10...
        "engine": engine,
    }
    time_of_last_api_call = _ms_since_epoch()

    return parameters, time_of_last_api_call


def wait_between_predictions(time_of_last_api_call, min_ms_between_api_calls):
    if (
        cur_time := _ms_since_epoch()
    ) <= time_of_last_api_call + min_ms_between_api_calls:
        ms_to_sleep = min_ms_between_api_calls - (cur_time - time_of_last_api_call)
        time.sleep(ms_to_sleep / 1000)
    time_of_last_api_call = _ms_since_epoch()


def predict_sample_openai_gpt(
    example,
    prompt,
    min_ms_between_api_calls: int = 5000,
    engine: str = "text-davinci-003",
    max_tokens: int = 100,
):
    parameters, time_of_last_api_call = set_openai_parameters(engine, max_tokens)
    parameters["prompt"] = prompt

    # OpenAI limits us to 3000 calls per minute:
    # https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits
    # that is why the default value of min_ms_between_api_calls is 20
    wait_between_predictions(time_of_last_api_call, min_ms_between_api_calls)

    response = openai.Completion.create(**parameters)

    if response is None:
        raise Exception("Response from OpenAI API is None.")

    # build output data
    prediction = dict()
    prediction["input"] = prompt
    prediction["prediction"] = response.choices[0].text.strip().strip(".")  # type:ignore

    # build output metadata
    metadata = example.copy()  # dict()
    metadata["logprobs"] = response.choices[0]["logprobs"]  # type:ignore
    # "finish_reason" is located in a slightly different location in opt
    if "opt" in engine:
        finish_reason = response.choices[0]["logprobs"][  # type:ignore
            "finish_reason"
        ]
    else:
        finish_reason = response.choices[0]["finish_reason"]  # type:ignore
    metadata["finish_reason"] = finish_reason
    if "opt" not in engine:
        # From the OpenAI API documentation it's not clear what "index" is, but let's keep it as well
        metadata["index"] = response.choices[0]["index"]  # type:ignore

    prediction["metadata"] = metadata

    return prediction

def predict_sample_openai_chatgpt(
    prompt,
    img_url,
    min_ms_between_api_calls: int = 10000,
    engine: str = "gpt-4o",
    max_tokens: int = 100,
):
    parameters, time_of_last_api_call = set_openai_parameters(engine, max_tokens)
    parameters["prompt"] = prompt

    # OpenAI limits us to 3000 calls per minute:
    # https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits
    wait_time = 10
    time.sleep(wait_time)
    try:
        response = openai.ChatCompletion.create(model=engine, messages=[{"role": "user", "content": [{"type": "text", "text":prompt},{
          "type": "image_url",
          "image_url": {"url": f"data:image/jpeg;base64,{img_url}"
}}]}], temperature=parameters['temperature'], top_p=parameters['top_p'])
    except openai.error.RateLimitError as e:
        wait_time = 10
        print(f"Rate limit reached. Waiting {wait_time} seconds.")
        time.sleep(wait_time)

        response = openai.ChatCompletion.create(model=engine, messages=[{"role": "user", "content": prompt}],
                                                temperature=parameters['temperature'], top_p=parameters['top_p'])

    if response is None:
        raise Exception("Response from OpenAI API is None.")

    # build output data
    prediction = dict()
    prediction["input"] = prompt
    prediction["prediction"] = response.choices[0].message['content']  # type:ignore

    return prediction

def gpt4_estimetion(url):
  prompt= f"""
Generate a caption for the provided image. If the image contains any nonsensical or uncommon elements, make sure to highlight them.
  """
  gpt4_prediction = predict_sample_openai_chatgpt(prompt,url)
  return gpt4_prediction['prediction']

# prepare WHOOPS! dataset

In [ ]:
!pip install -q git-lfs
!git clone https://huggingface.co/spaces/nlphuji/whoops-explorer-analysis
!pip install -q datasets
from datasets import load_dataset
# import gradio as gr
import os
import random

wmtis = load_dataset("nlphuji/wmtis-identify")['test'][73:]

fatal: destination path 'whoops-explorer-analysis' already exists and is not an empty directory.


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for nlphuji/wmtis-identify contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nlphuji/wmtis-identify
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Run captions generation task on strange, normal, natural images

In [ ]:
import base64
from io import BytesIO
from openai.error import InvalidRequestError

gpt4_captions ={'natural_caption':[],'normal_caption':[],'strange_caption':[]}
for record in wmtis:
  print(record)
  natural_image = record['natural_image']
  normal_image = record['normal_image']
  strange_image = record['strange_image']
  print(record)

  buffered = BytesIO()
  natural_image.save(buffered, format="PNG")
  natural_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
  try:
    natural_caption = gpt4_estimetion(natural_str)
    print(natural_caption)
    gpt4_captions['natural_caption'].append(natural_caption)
  except InvalidRequestError as e:
    gpt4_captions['natural_caption'].append(f'error: {natural_caption}')
    print(f"Failed to get caption: {e}")


  buffered = BytesIO()
  normal_image.save(buffered, format="PNG")
  normal_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
  try:
    normal_caption = gpt4_estimetion(normal_str)
    print(normal_caption)
    gpt4_captions['normal_caption'].append(normal_caption)
  except InvalidRequestError as e:
    gpt4_captions['normal_caption'].append(f'error: {normal_caption}')
    print(f"Failed to get caption: {e}")

  buffered = BytesIO()
  strange_image.save(buffered, format="PNG")
  strange_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
  try:
    strange_caption = gpt4_estimetion(strange_str)
    print(strange_caption)
    gpt4_captions['strange_caption'].append(strange_caption)
  except InvalidRequestError as e:
    gpt4_captions['strange_caption'].append(f'error: {strange_caption}')
    print(f"Failed to get caption: {e}")


In [ ]:
# save the outputs to a csv files
import pandas as pd
natural_df = pd.DataFrame(gpt4_captions['natural_caption'])
natural_df.to_csv('natural_caption_improvedPrompt.csv', index=False)  # index=False to avoid writing row numbers

normal_df = pd.DataFrame(gpt4_captions['normal_caption'])
normal_df.to_csv('normal_caption_improvedPrompt.csv', index=False)  # index=False to avoid writing row numbers

strange_df = pd.DataFrame(gpt4_captions['strange_caption'])
strange_df.to_csv('strange_caption_improvedPrompt.csv', index=False)  # index=False to avoid writing row numbers